In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("spark-dataframe").getOrCreate()

## 파일이나 다른 데이터 소스로 부터 스파크 데이터 프레임 만드는 방법
- spark.read.xxx(DataSource 경로)

In [3]:
pwd

'c:\\Users\\apfhd\\SparkExam'

In [4]:
directory='c://Users//apfhd//SparkExam//data//titanic_train.csv'

In [5]:
# pandas로 csv 불러오기 - 행렬 병렬 처리
import pandas as pd
titanic_pdf = pd.read_csv(f'{directory}',header="infer")# header="infer" 타입을 자동 유추해준다.
titanic_pdf.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
# Spark로 csv 데이터 불러오기 - 병렬 분산 처리
titanic_sdf=spark.read.csv(f'file:///{directory}',header=True, inferSchema=True) 
titanic_sdf
# header=True : csv 기록된 컬럼 정보 포함
# inferschema= True : 스키마 유추한다.

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: string, Embarked: string]

In [7]:
titanic_sdf.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+-------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch| Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+-------+-------+-----+--------+
|        892|       0|     3|    Kelly, Mr. James|  male|34.5|    0|    0| 330911| 7.8292| null|       Q|
|        893|       1|     3|Wilkes, Mrs. Jame...|female|47.0|    1|    0| 363272|    7.0| null|       S|
|        894|       0|     2|Myles, Mr. Thomas...|  male|62.0|    0|    0| 240276| 9.6875| null|       Q|
|        895|       0|     3|    Wirz, Mr. Albert|  male|27.0|    0|    0| 315154| 8.6625| null|       S|
|        896|       1|     3|Hirvonen, Mrs. Al...|female|22.0|    1|    1|3101298|12.2875| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+-------+-------+-----+--------+
only showing top 5 rows



# Pandas DataFrame 과 Spark DataFrame 의 주요 차이
- pandas
    - 싱글로드
    - ``행렬 병렬처리``
- spark 
    - 여러대의 클러스트
    - ``병렬 분산처리`` 

### Spark DataFrame은 SQL 연산과 비슷한 연산자를 제공
- spark_dataframe.select('컬럼명')
- spark_dataframe.select('컬럼명').filter(...)
    - filter : where절에 해당!
- spark_dataframe.groupBy('컬럼명').count()
- spark_dataframe.withColumns('컬럼명',...)

### Spark Dataframe의 연산의 특징
- Spark Dataframe의 연산은 대부분 새로운 Dataframe 객체를 반환하는 형태로 구성  
- 특히 Dataframe 객체에 직접 수정을 허용하지 않는다.  
    - Spark Dataframeeh RDD의 immutable 특징을 그대로 가져간다.  
- spark dataframe은 RDD기반 이루어져있다.  
==> ``spark dataframe 객체에 직접 수정을 허용하지 않는다``  
* <u>pandas에서는 inplace를 통해 직접 수정이 가능하다.</u>  
    <br>

#### 비교  
1. drop  
    - pandas
        - padnas_Dataframe.drop('컬럼명',axis=1,inplace=True) 호출하면 pandas_Dataframe 객체 자체에서 "컬럼명'을 drop 시킨다.  
    - ``Spark Dataframe``  
        - Spark Dataframe_new =  Spark_Dataframe.drop('컬럼명')과 같이   
        ``inplace 인자가 아예없음 ==> 새로운 변수로 받아줘야한다.``  
        <br>
2. 새로운 컬럼 만들기
    - Pandas
        - 특정 컬럼 값을 가져오거나, 새로운 컬럼을 만들기 위해서 사용했다.  
        - pandas_dataframe['new_column'] = pandas_dataframe['column'] * 10  
    - ``Spark Dataframe``  
        - withColumns() 메소드를 활용  
        - update 효과가 있다.  
        - Spark Dataframe.withColumns('new_column',col('column')*10)  
        - ``withColumns(), filters() 메소드에서 컬럼을 지정하기 위해서만 사용됨 -> [] 사용``  
        <br>
3. 컬럼에 접근하기  
    - pandas  
        - pandas_dataframe.drop(['컬럼1','컬럼2'])  
    - ``Spark Dataframe``  
        - Spark_Dataframe.drop('컬럼1','컬럼2')  
        <br>
4. Head 비교
    - pandas  
        - head : 데이터 프레임
    - ``Spark ``
        - head : Row object가 들어있는 list
        - ``limit.show()`` : 데이터 프레임
        <br>
5. info 정보
    - pandas
        - 컬럼명, Data Type, ``not null`` 건수도 나온다.
    - ``Spark``
        - ``describe()`` 대신하나 컬럼명:Data Type 만 출력
        - ``not null`` 을 위해 별도의 SQL 쿼리가 필요

In [8]:
titanic_pdf.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [9]:
titanic_sdf.head(5) ## 이런모양은 쿼리를 칠수 없다. 데이터 프레임이 아니다. 데이터 프레임 함수를 칠수 없다.

[Row(PassengerId=892, Survived=0, Pclass=3, Name='Kelly, Mr. James', Sex='male', Age=34.5, SibSp=0, Parch=0, Ticket='330911', Fare=7.8292, Cabin=None, Embarked='Q'),
 Row(PassengerId=893, Survived=1, Pclass=3, Name='Wilkes, Mrs. James (Ellen Needs)', Sex='female', Age=47.0, SibSp=1, Parch=0, Ticket='363272', Fare=7.0, Cabin=None, Embarked='S'),
 Row(PassengerId=894, Survived=0, Pclass=2, Name='Myles, Mr. Thomas Francis', Sex='male', Age=62.0, SibSp=0, Parch=0, Ticket='240276', Fare=9.6875, Cabin=None, Embarked='Q'),
 Row(PassengerId=895, Survived=0, Pclass=3, Name='Wirz, Mr. Albert', Sex='male', Age=27.0, SibSp=0, Parch=0, Ticket='315154', Fare=8.6625, Cabin=None, Embarked='S'),
 Row(PassengerId=896, Survived=1, Pclass=3, Name='Hirvonen, Mrs. Alexander (Helga E Lindqvist)', Sex='female', Age=22.0, SibSp=1, Parch=1, Ticket='3101298', Fare=12.2875, Cabin=None, Embarked='S')]

In [10]:
# 컬럼의 상세정보
titanic_sdf.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [11]:
# Not null 카운트 확인하기
from pyspark.sql.functions import count, isnan, when, col

titanic_sdf.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in titanic_sdf.columns]).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0| 86|    0|    0|     0|   1|  327|       0|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



## Spark의 select()

In [12]:
dict_01 = {'Name': ['상권', '동영','재준','성국', '성동'],
           'Year': [2012, 2013, 2013, 2012, 2012],
           'Gender': ['Male', 'Male', 'Male', 'Female', 'Male']
          }

In [13]:
data_pdf= pd.DataFrame(dict_01)
data_sdf = spark.createDataFrame(data_pdf) #판다스 데이터 프레임을 스파크 데이터 프레임으로 만들기
# data_sdf 테이블로 생각하면 된다.

In [14]:
data_pdf[['Name',"Year"]]

,Name,Year
0,상권,2012
1,동영,2013
2,재준,2013
3,성국,2012
4,성동,2012


In [15]:
# RDD 처럼 나온다. 원본데이터가 변형되었으니깐!
data_sdf.select("Name")

DataFrame[Name: string]

In [16]:
# 컬럼 속성으로 지정하여 select의 인자로 사용이 가능하다
data_sdf.Name

Column<'Name'>

In [17]:
# 컬럼 속성으로 지정하여 select의 인자로 사용이 가능하다
data_sdf.select(data_sdf.Name,data_sdf.Year).show()

+----+----+
|Name|Year|
+----+----+
|상권|2012|
|동영|2013|
|재준|2013|
|성국|2012|
|성동|2012|
+----+----+



In [18]:
from pyspark.sql.functions import col # 명시적으로 컬럼명을 지정할 수 있는 함수
# withcolumns 와 filter와 굉장히 많이 활용된다. 데이터를 가지고 와서 수정할 수 있게끔 한다.
# col() 함수를 이용하며 명시적으로 컬럼명을 지정할 수 있음.
data_sdf.select(col('Name'),col("Year")).show()

+----+----+
|Name|Year|
+----+----+
|상권|2012|
|동영|2013|
|재준|2013|
|성국|2012|
|성동|2012|
+----+----+



In [19]:
from pyspark.sql.functions import upper,lower,col

In [20]:
# select()에서 컬럼 데이터를 가공 후 생성 가능하다.
data_sdf.select("*",upper(col("Gender"))).show() # select *, upper(Gender) from data_sdf

+----+----+------+-------------+
|Name|Year|Gender|upper(Gender)|
+----+----+------+-------------+
|상권|2012|  Male|         MALE|
|동영|2013|  Male|         MALE|
|재준|2013|  Male|         MALE|
|성국|2012|Female|       FEMALE|
|성동|2012|  Male|         MALE|
+----+----+------+-------------+



In [21]:
data_sdf.select("*",upper(col("Gender")).alias("Cap_Gender")).show()

+----+----+------+----------+
|Name|Year|Gender|Cap_Gender|
+----+----+------+----------+
|상권|2012|  Male|      MALE|
|동영|2013|  Male|      MALE|
|재준|2013|  Male|      MALE|
|성국|2012|Female|    FEMALE|
|성동|2012|  Male|      MALE|
+----+----+------+----------+



## Spark DataFrame의 filter 메소드 
- `filter()` 는 `SQL WHERE 절`과 비슷 DataFrame 내의 특정 조건을 만족하는 레코드를 DataFrame으로 변환
- filter() 내의 조건 컬럼은 컬럼 속성으로 지정 가능/ 조건문 자체는 ``SQL과 유사한 문자열로 지정``할 수 있다.
- where() 메소드는 filter의 alias 함수이며, where 직관적인 동일성을 간주하기 위해 생성
    - ``filter() == where()``
- 복합조건 and or는 각각 &, | 로 사용

In [22]:
dict_01 = {'Name': ['상권', '동영','재준','성국', '성동'],
           'Year': [2012, 2013, 2013, 2012, 2012],
           'Gender': ['Male', 'Male', 'Male', 'Female', 'Male']
          }

In [23]:
# dict -> pandas DataFrame 변환
data_pdf=pd.DataFrame(dict_01)
data_pdf

,Name,Year,Gender
0,상권,2012,Male
1,동영,2013,Male
2,재준,2013,Male
3,성국,2012,Female
4,성동,2012,Male


In [24]:
# pandas DataFrame -> Spark DataFrame
data_sdf=spark.createDataFrame(data_pdf)
data_sdf

DataFrame[Name: string, Year: bigint, Gender: string]

In [25]:
data_sdf.filter('Name'== '상권') # 조건을 표현하기 위한 문자열이 들어가야한다.

TypeError: condition should be string or Column

In [ ]:
# () -> SQL의 where 절 처럼 사용해야한다.
data_sdf.filter("Name='상권'").show() # select * from data_sdf where Name='민호'

+----+----+------+
|Name|Year|Gender|
+----+----+------+
|상권|2012|  Male|
+----+----+------+



## 복합 조건 and(&) or(|)

In [ ]:
# Gender가 Male이고 Year가  2011년 이상인 사람의 모든 정보
data_sdf.filter((data_sdf["Gender"]=="Male")&(col("Year")>2011)).show()
# 복합 조건 시에 연산우선순위에 따라  ()로 다 묶어주는 것이 안전하다.

+----+----+------+
|Name|Year|Gender|
+----+----+------+
|상권|2012|  Male|
|동영|2013|  Male|
|재준|2013|  Male|
|성동|2012|  Male|
+----+----+------+



and or를 같이 사용하게될 때 ``and가 우선순위가 높다.``  
- or and -> and 먼저 실행    
- (or) and  -> or 먼저 실행(가로 덕분에)    

### 문자열 컬럼 like 조건 수행
### ``특정 컬럼을 선택 -> 조건 수행``

In [ ]:
# 문자열 컬럼 like 조건 수행
# 특정 컬럼을 선택 -> 조건 수행
data_sdf.filter(col("Name").like("성%")).show()

+----+----+------+
|Name|Year|Gender|
+----+----+------+
|성국|2012|Female|
|성동|2012|  Male|
+----+----+------+



In [ ]:
data_sdf.filter("Name like '성%'").show()

+----+----+------+
|Name|Year|Gender|
+----+----+------+
|성국|2012|Female|
|성동|2012|  Male|
+----+----+------+



In [ ]:
data_sdf.filter("upper(Gender) like('%A%')").show() # pyspark.sql.fuctions 에서 가져온 upper 함수.

+----+----+------+
|Name|Year|Gender|
+----+----+------+
|상권|2012|  Male|
|동영|2013|  Male|
|재준|2013|  Male|
|성국|2012|Female|
|성동|2012|  Male|
+----+----+------+



In [ ]:
from pyspark.sql.functions import upper
data_sdf.filter(upper(data_sdf["Gender"]).like("%A%")).show()

+----+----+------+
|Name|Year|Gender|
+----+----+------+
|상권|2012|  Male|
|동영|2013|  Male|
|재준|2013|  Male|
|성국|2012|Female|
|성동|2012|  Male|
+----+----+------+



## Spark DataFrame의 `orderBy()` 알아보기

In [ ]:
from pyspark.sql.functions import col

print("orderBy에 컬럼명을 문자열로 지정하고 내림 차순 정렬")
# titanic_sdf.orderBy("Name",ascending=False).show()

# 직접 컬럼을 선택해서 내림 차순 정렬
# 1번 방식)titanic_sdf.orderBy(titanic_sdf["Name"],ascending=True).show()
# 2번 방식)titanic_sdf.orderBy(titanic_sdf.Name, ascending=True).show()
# 3번 방식)
titanic_sdf.orderBy(col("Name"), ascending=True).show(5) #  이게 제일 추천

orderBy에 컬럼명을 문자열로 지정하고 내림 차순 정렬
+-----------+--------+------+--------------------+------+----+-----+-----+----------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|    Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------+-------+-----+--------+
|        911|       1|     3|"Assaf Khalil, Mr...|female|45.0|    0|    0|      2696|  7.225| null|       C|
|        941|       1|     3|"Coutts, Mrs. Wil...|female|36.0|    0|    2|C.A. 37671|   15.9| null|       S|
|        925|       1|     3|"Johnston, Mrs. A...|female|null|    1|    2|W./C. 6607|  23.45| null|       S|
|        927|       0|     3|"Katavelas, Mr. V...|  male|18.5|    0|    0|      2682| 7.2292| null|       C|
|       1141|       1|     3|"Khalil, Mrs. Bet...|female|null|    1|    0|      2660|14.4542| null|       C|
+-----------+--------+------+--------------------+------+----+-----+-----+----------+-------+--

In [ ]:
titanic_sdf.orderBy("Pclass","Name",ascending=False).show(5)

+-----------+--------+------+--------------------+----+----+-----+-----+--------+-----+-----+--------+
|PassengerId|Survived|Pclass|                Name| Sex| Age|SibSp|Parch|  Ticket| Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+----+----+-----+-----+--------+-----+-----+--------+
|       1084|       0|     3|van Billiard, Mas...|male|11.5|    1|    1|A/5. 851| 14.5| null|       S|
|       1236|       0|     3|van Billiard, Mas...|male|null|    1|    1|A/5. 851| 14.5| null|       S|
|       1152|       0|     3|de Messemaeker, M...|male|36.5|    1|    0|  345572| 17.4| null|       S|
|       1063|       0|     3| Zakarian, Mr. Ortin|male|27.0|    0|    0|    2670|7.225| null|       C|
|       1028|       0|     3|Zakarian, Mr. Map...|male|26.5|    0|    0|    2656|7.225| null|       C|
+-----------+--------+------+--------------------+----+----+-----+-----+--------+-----+-----+--------+
only showing top 5 rows



In [ ]:
titanic_sdf.orderBy(col("Pclass").asc(),col("Name").desc()).show(5) 

+-----------+--------+------+--------------------+------+----+-----+-----+--------+-------+-------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|  Ticket|   Fare|  Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+-------+-------+--------+
|       1263|       1|     1|Wilson, Miss. Hel...|female|31.0|    0|    0|   16966|  134.5|E39 E41|       C|
|        915|       0|     1|Williams, Mr. Ric...|  male|21.0|    0|    1|PC 17597|61.3792|   null|       C|
|       1123|       1|     1|Willard, Miss. Co...|female|21.0|    0|    0|  113795|  26.55|   null|       S|
|       1110|       1|     1|Widener, Mrs. Geo...|female|50.0|    1|    1|  113503|  211.5|    C80|       C|
|       1299|       0|     1|Widener, Mr. Geor...|  male|50.0|    1|    1|  113503|  211.5|    C80|       C|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+-------+-------+--------+
only showing top 5 

In [ ]:
# 데이터 추출하고 (Pclass, Name을 select)하고 정렬
# select Pclass,Name from titanic_sdf order by Pclass asc, Name desc 
# 데이터 선택하고 내가 필요한 부분만 뽑아오는 방식
titanic_sdf.select(col("Pclass"),col("name")).orderBy(col("Pclass").asc(),col("name").desc()).show(5)

+------+--------------------+
|Pclass|                name|
+------+--------------------+
|     1|Wilson, Miss. Hel...|
|     1|Williams, Mr. Ric...|
|     1|Willard, Miss. Co...|
|     1|Widener, Mrs. Geo...|
|     1|Widener, Mr. Geor...|
+------+--------------------+
only showing top 5 rows



In [ ]:
# 데이터를 정렬하고 추출 - 대량의 데이터라면 여기가 유리하다. 셔플링작업을 최소화 할 수 있다.
# select Pclass, Name from (select * from titanic_sdf order by Pclass asc,Name desc)
titanic_sdf.orderBy(col("Pclass").asc(),col("Name").desc()).select(col("Pclass"),col("Name")).show(5)

+------+--------------------+
|Pclass|                Name|
+------+--------------------+
|     1|Wilson, Miss. Hel...|
|     1|Williams, Mr. Ric...|
|     1|Willard, Miss. Co...|
|     1|Widener, Mrs. Geo...|
|     1|Widener, Mr. Geor...|
+------+--------------------+
only showing top 5 rows



## Spark DataFrame의 groupby()
    - pandas
        - groupby(by='컬럼명') -> DataFrameGroupBy 객체가 반환 + aggregation
    - Spark
        - groupby(by='컬럼명') -> GroupedData 객체가 반환 + aggregation

In [29]:
titanic_pdf_groupby=titanic_pdf.groupby(by="Pclass")
titanic_pdf_groupby.count() # 판다스에서는 count()에 값을 지정하지 않아도 각 컬럼별 개수가 나오지만 Spark는 특정 컬럼 지정해줘야한다.

,PassengerId,Survived,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
Pclass,,,,,,,,,,,
1,107,107,107,107,98,107,107,107,107,80,107
2,93,93,93,93,88,93,93,93,93,7,93
3,218,218,218,218,146,218,218,218,217,4,218


In [32]:
# 각 컬럼에 대한 다른 집계
agg_format={
    "Age":"max",
    "SibSp":"sum",
    "Fare":'mean'
}

titanic_pdf_groupby.agg(agg_format)

,Age,SibSp,Fare
Pclass,,,
1,76.0,51,94.280297
2,63.0,35,22.202104
3,60.5,101,12.459678


In [33]:
# pandas
# value_counts : pandas에서 series 안쪽 값 구하기
titanic_pdf["Pclass"].value_counts()

3    218
1    107
2     93
Name: Pclass, dtype: int64

In [34]:
# Spark
titanic_sdf.groupby("Pclass").count().show()
# GroupedData 객체에 count()외에 min(),Max(),avg(),sum() 등 다양한 aggregation 메소드를 호출하여
# group by, aggregation 결과 DataFrame이 반환

+------+-----+
|Pclass|count|
+------+-----+
|     1|  107|
|     3|  218|
|     2|   93|
+------+-----+



In [41]:
# Pclass 카운트 내림 차순 정렬
titanic_sdf.orderBy(col("Pclass").desc()).show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|  Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+------+-----+--------+
|        898|       1|     3|Connolly, Miss. Kate|female|30.0|    0|    0|          330972|7.6292| null|       Q|
|        910|       1|     3|Ilmakangas, Miss....|female|27.0|    1|    0|STON/O2. 3101270| 7.925| null|       S|
|        900|       1|     3|Abrahim, Mrs. Jos...|female|18.0|    0|    0|            2657|7.2292| null|       C|
|        893|       1|     3|Wilkes, Mrs. Jame...|female|47.0|    1|    0|          363272|   7.0| null|       S|
|        901|       0|     3|Davies, Mr. John ...|  male|21.0|    2|    0|       A/4 48871| 24.15| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+--------------

In [44]:
# Pclass별로 카운트 내림 차순 정렬
titanic_sdf.groupBy("Pclass").count().orderBy("count", ascending=False).show()

+------+-----+
|Pclass|count|
+------+-----+
|     3|  218|
|     1|  107|
|     2|   93|
+------+-----+



### GroupedData에서 aggregation 메소드를 호출 할때는 ``오직 문자열 컬럼명만`` 가능   
- ex) ``max("Age") -> ok`` / ``max(col("Age")) -> Error``
- (컬럼형 인자는 오류 발생)

In [45]:
# age 만 확인하기
titanic_sdf.groupBy("Pclass").max("Age").show()

+------+--------+
|Pclass|max(Age)|
+------+--------+
|     1|    76.0|
|     3|    60.5|
|     2|    63.0|
+------+--------+



In [46]:
# age 만 확인하기
titanic_sdf.groupBy("Pclass").max(col("Age")).show()

TypeError: Column is not iterable

In [47]:
# 여러개 agg 할 경우 agg(개별 agg함수 작성)
titanic_sdf.groupBy("Pclass").agg(max("Age").alias("Max_age"),sum("Age").alias("Total")).show()

+------+-------+-------+
|Pclass|Max_age|  Total|
+------+-------+-------+
|     1|   76.0| 4010.0|
|     3|   60.5|3508.08|
|     2|   63.0|2532.42|
+------+-------+-------+

